You can write **your own** `.NET HTTP server` with the `HttpListener` class.

In [ ]:
//simple server that listens on port 51111
//waits for a single client
//request, and then returns a one-line reply

using System.Net;
using System.Net.Http;
using System.Text;
using System.IO;


public class SimpleHttpServer : IDisposable
{
    private readonly HttpListener listener = new HttpListener();

    public SimpleHttpServer() => ListenAsync();

    private async void ListenAsync()
    {
        listener.Prefixes.Add("http://localhost:51111/MyApp/"); // Listen on
        listener.Start(); // port 51111
                          // Await a client request:
        HttpListenerContext context = await listener.GetContextAsync();
        // Respond to the request:
        string msg = "You asked for: " + context.Request.RawUrl;
        context.Response.ContentLength64 = Encoding.UTF8.GetByteCount(msg);
        context.Response.StatusCode = (int)HttpStatusCode.OK;
        using (Stream s = context.Response.OutputStream)
        using (StreamWriter writer = new StreamWriter(s))
            await writer.WriteAsync(msg);
    }

    public void Dispose() => listener.Close();
}

using (var server = new SimpleHttpServer())
{
    // Make a client request:
    Console.WriteLine (await new HttpClient().GetStringAsync
        ("http://localhost:51111/MyApp/Request.txt"));
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p><code>HttpListener</code> چگونه در ویندوز کار می‌کند و چه مزایایی دارد:</p>
<ol><li><p><strong>استفاده از Windows HTTP Server API</strong>:</p><ul><li><code>HttpListener</code> به جای استفاده از اشیاء <code>Socket</code> در .NET، از Windows HTTP Server API استفاده می‌کند.</li><li>این امر باعث می‌شود که <code>HttpListener</code> بتواند به طور مستقیم با امکانات بومی ویندوز برای مدیریت درخواست‌های HTTP تعامل داشته باشد.</li></ul></li><li><p><strong>گوش دادن به یک پورت با پیشوندهای مختلف</strong>:</p><ul><li>این معماری اجازه می‌دهد که چندین برنامه بتوانند بر روی یک پورت و IP مشابه گوش دهند، به شرطی که هر یک پیشوند آدرس (prefix) متفاوتی را ثبت کنند.</li><li>به عنوان مثال، در کد ما، پیشوند <code>http://localhost:51111/MyApp/</code> ثبت شده است. برنامه دیگری می‌تواند با پیشوند <code>http://localhost:51111/AnotherApp/</code> بر روی همان پورت و IP گوش دهد.</li></ul></li><li><p><strong>مزیت در محیط‌های شرکتی</strong>:</p><ul><li>این ویژگی به ویژه در محیط‌های شرکتی مفید است، زیرا باز کردن پورت‌های جدید در فایروال‌های شرکتی ممکن است با مشکلات سیاسی و امنیتی مواجه شود.</li><li>با این روش، برنامه‌های مختلف می‌توانند از پورت‌های موجود استفاده کنند بدون نیاز به درخواست باز کردن پورت‌های جدید.</li></ul></li></ol>

<p> با استفاده از مکانیزم <code>HttpListener</code> و پشتیبانی از Windows HTTP Server API، شما می‌توانید یک وب‌سایت در IIS تعریف کنید که به درخواست‌هایی مثل <code>https://myserverdomain.com/iisapp</code> گوش دهد و همزمان یک اپلیکیشن جداگانه‌ای بنویسید که با استفاده از <code>HttpListener</code> به درخواست‌هایی که به URL دیگری مانند <code>https://myserverdomain.com/myapp</code> ارسال می‌شوند، گوش دهد.</p>
</div>

In [ ]:
using System;
using System.Net;
using System.Text;
using System.Threading.Tasks;
using System.IO;

public class SimpleHttpServer : IDisposable
{
    private readonly HttpListener listener = new HttpListener();

    public SimpleHttpServer() => ListenAsync();

    private async void ListenAsync()
    {
        listener.Prefixes.Add("https://myserverdomain.com/myapp/");
        listener.Start();

        while (true)
        {
            // Await a client request:
            HttpListenerContext context = await listener.GetContextAsync();
            // Respond to the request:
            string msg = "You asked for: " + context.Request.RawUrl;
            context.Response.ContentLength64 = Encoding.UTF8.GetByteCount(msg);
            context.Response.StatusCode = (int)HttpStatusCode.OK;
            using (Stream s = context.Response.OutputStream)
            using (StreamWriter writer = new StreamWriter(s))
                await writer.WriteAsync(msg);
        }
    }

    public void Dispose() => listener.Close();
}

class Program
{
    static void Main()
    {
        using (var server = new SimpleHttpServer())
        {
            Console.WriteLine("Server is running... Press Enter to stop.");
            Console.ReadLine();
        }
    }
}
